<a href="https://www.kaggle.com/code/thierrymasters/salifort-motors-employee-retention-prediction?scriptVersionId=144605639" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Data-driven suggestions for Salifort Motors HR**

## Business Understanding

The HR department at Salifort Motors is taking positive steps to enhance employee satisfaction levels, which is encouraging. To understand why an employee might leave the company, it's crucial to review the data collected by HR and create a model that can accurately predict the likelihood of an employee departing. This approach enables us to determine the reasons behind their departure and take corrective measures. Since finding, interviewing, and hiring new employees can be time-consuming and costly, increasing employee retention is essential. The company can benefit in various ways by doing so.

## Data Understanding

The dataset that you'll be using in this lab contains 15,000 rows and 10 columns for the variables listed below. 




**source:** For more information about the data, refer to its source on [Kaggle](https://www.kaggle.com/datasets/mfaisalqureshi/hr-analytics-and-job-prediction?select=HR_comma_sep.csv).

Variable  |Description |
-----|-----|
satisfaction_level|Employee-reported job satisfaction level [0&ndash;1]|
last_evaluation|Score of employee's last performance review [0&ndash;1]|
number_project|Number of projects employee contributes to|
average_monthly_hours|Average number of hours employee worked per month|
time_spend_company|How long the employee has been with the company (years)
Work_accident|Whether or not the employee experienced an accident while at work
left|Whether or not the employee left the company
promotion_last_5years|Whether or not the employee was promoted in the last 5 years
Department|The employee's department
salary|The employee's salary (U.S. dollars)

### Import packages

In [ ]:
# Import packages

# Functional packages
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Build model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Metrics and other functions
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import plot_tree

# For display all columns in dataframes
pd.set_option('display.max_columns', None)

# Saving model
import pickle

# Warnings 
import warnings
warnings.filterwarnings("ignore")

### Load dataset


In [ ]:
# Load dataset into a dataframe
df_raw= pd.read_csv("/kaggle/input/hr-analytics-and-job-prediction/HR_comma_sep.csv")

# Display first few rows of the dataframe
df_raw.head()

## Initial EDA and data cleaning

- Understand your variables
- Clean your dataset (missing data, redundant data, outliers)



In [ ]:
# Basic information about the data
df_raw.info()

In [ ]:
# Descriptive statistics about the data
df_raw.describe()

### Rename columns

We inspect the columns names to see if the are names that stand out or need to be modified

In [ ]:
# Display all column names
df_raw.columns

In [ ]:
# Rename columns
df_raw = df_raw.rename(columns={"Work_accident" : "work_accident",
                                "time_spend_company" : "tenure",
                                "average_montly_hours" : "average_monthly_hours",
                                "Department" : "department"})

# Display all column names after the update
df_raw.columns

### Missing values

In [ ]:
# Check for missing values
df_raw.isna().sum()

There are no missing values.

### Duplicates

In [ ]:
# Check for duplicates
duplucates = df_raw.duplicated()
print(f'Duplicates : {duplucates.sum()} \nPercentage : {duplucates.sum() / len(df_raw) * 100 :.2f} %')

We have identified 3008 duplicate entries, which account for 20% of the total data set. It is important to take this proportion into consideration when managing the duplicates in order to maintain the integrity of the data. Let's begin by examining a few of the duplicate entries.

In [ ]:
# Inspect some rows containing duplicates as needed
df_raw[duplucates].head()

Let's drop the duplicates.

In [ ]:
# Drop duplicates and save resulting dataframe in a new variable as needed
df = df_raw.drop_duplicates(keep='first')

# Display first few rows of new dataframe as needed
df.head()

### Outliers

First, we examine the boxpot of the `tenure` column for any outliers.

In [ ]:
# Create a boxplot to visualize distribution of `tenure` and detect any outliers
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.boxplot(x='tenure', data=df)
ax.set_title("Boxplot to detect outlier for Tenure")
plt.show;

Then, we define a function to calculate the number of outliers in a selected column.

In [ ]:
# Determine the number of rows containing outliers

def num_outliers(column):
    """
        This function takes as parameter a column of the dataset and does the following:
         - computes its 25th and 75th percentiles
         - determine the interquantile range
         - define the upper and lower limits for non_outlier values
         - count and print the number of outlier
        
        Formulas:
        interquantile range (iqr): 75th percentile - 25th percentile
        upper limit : 75th percentile + 1.5 * iqr
        lower limit : 25th percentile - 1.5 * iqr

    """

    # Compute 25th percentile
    percent_25 = df[column].quantile(0.25)

    # Compute 75th percentile
    percent_75 = df[column].quantile(0.75)

    # Compute interquantile range
    iqr = percent_75 - percent_25

    # Define upper and lower limits for non-oulier values in column
    upper_limit = percent_75 + 1.5 * iqr
    lower_limit = percent_25 - 1.5 * iqr
    print(f'Upper Limit : {upper_limit}')
    print(f'Lower Limit : {lower_limit}')

    # Identify the subset of data containing outliers
    outliers = df[(df[column] > upper_limit) | (df[column] < lower_limit)]

    # Count number of rows with outliers
    print(f"Number of Outliers in '{column}' : {len(outliers)} ")


We can now inquire the number of outliers in `tenure` column.

In [ ]:
# Number of rows with outliers in 'tenure'
num_outliers('tenure')

Different models have varying sensitivity to outliers. Once we reach the model-building stage, we will assess the need to eliminate outliers depending on the specific model we opt to utilise.

## Data Exploration (Continue EDA)

we will begin by understanding how many employees left and what percentage of all employees this figure represents.

In [ ]:
# Get numbers of people who left vs. stayed
num_left = len(df[df['left'] == 1])
num_stayed = len(df) - num_left
print(f"Number of employees that left : {num_left}")
print(f"Number of employees that stayed : {num_stayed}")

# Get percentages of people who left vs. stayed
print(f"Percentage leaving : {num_left / len(df) * 100 :.2f} %")
print(f"Percentage staying : {num_stayed / len(df) * 100 :.2f} %")

There are 16.6% employees leaving the company. That is close to a fifth of total number of employees.

### Data visualisations

In [ ]:
# Set figure and axes
fig, ax = plt.subplots(1, 2, figsize=(22, 8))

# Create boxplot showing `average_monthly_hours` distributions for `number_project`, comparing employees who stayed versus those who left
sns.boxplot(x="average_monthly_hours", y="number_project", data=df, hue='left', orient='h', ax=ax[0])
ax[0].invert_yaxis()
ax[0].set_xlabel("Average Monthly Hours")
ax[0].set_ylabel("Number of Projects")
ax[0].set_title("Monthly Hours by number of projects")

# Create histogram showing distribution of `number_project`, comparing employees who stayed versus those who left
sns.histplot(x='number_project', data=df, hue='left', multiple='dodge', shrink=3, ax=ax[1])
ax[1].set_xlabel("Number of Projects")
ax[1].set_title("Distribution of Number of Projects")

# Display the plots
plt.show();

Employees who work on more projects also tend to work longer hours. This is evident from the increasing mean hours for both groups (stayed and left) as the number of projects worked increases. However, upon closer inspection, two distinct groups of employees left the company. The first group worked considerably less than their peers with the same number of projects, and the second group worked much more. The former group may have been fired or were already out the door, while the latter likely quit. It's reasonable to assume that employees in the second group were significant contributors to their projects.

Interestingly, everyone with seven projects left the company, and the interquartile ranges of this group and those who left with six projects were much higher than any other group. The data suggests that the optimal number of projects for employees to work on is 3-4, as the ratio of left/stayed is much smaller for these cohorts.

Assuming a work week of 40 hours and two weeks of vacation per year, the average number of working hours per month for employees working Monday-Friday is 166.67 hours per month. Except for employees who worked on two projects, every group worked considerably more than this, indicating that employees at this company may be overworked.

We will now investigate the average monthly hours versus the satisfaction levels

In [ ]:
# Create a plot as needed
# Create scatterplot of `average_monthly_hours` versus `satisfaction_level`, comparing employees who stayed versus those who left
plt.figure(figsize=(16, 9))
sns.scatterplot(data=df, x='average_monthly_hours', y='satisfaction_level', hue='left', alpha=0.4)
plt.axvline(x=166.67, color='#ff6361', label='166.67 hrs./mo.', ls='--')
plt.legend(labels=['166.67 hrs./mo.', 'left', 'stayed'])
plt.title('Monthly hours by last evaluation score', fontsize='14');

Looking at the chart provided, many employees worked between 240 and 315 hours per month. To put this into perspective, that's over 75 hours per week for an entire year. This may have contributed to their low levels of satisfaction.

Additionally, there is another group of individuals who left the company, and they had more typical working hours. However, their satisfaction levels were still relatively low, hovering around 0.4. It's difficult to say why they may have left, but it's possible that they felt pressured to work longer hours due to their peers working more.

On the other hand, a group of employees worked between 210 and 280 hours per month, and they had higher satisfaction levels ranging from 0.7 to 0.9. However, the strange distribution shape of the data suggests that there may have been some manipulation or synthetic data involved.

Next, we examine salary levels for different tenures.

In [ ]:
# Set figure and axes
fig, ax = plt.subplots(1, 2, figsize = (22,8))

# Define short-tenured employees
tenure_short = df[df['tenure'] < 7]

# Define long-tenured employees
tenure_long = df[df['tenure'] > 6]

# Plot short-tenured histogram
sns.histplot(data=tenure_short, x='tenure', hue='salary', discrete=1, 
             hue_order=['low', 'medium', 'high'], multiple='dodge', shrink=.5, ax=ax[0])
ax[0].set_title('Salary histogram by tenure: short-tenured people', fontsize='14')

# Plot long-tenured histogram
sns.histplot(data=tenure_long, x='tenure', hue='salary', discrete=1, 
             hue_order=['low', 'medium', 'high'], multiple='dodge', shrink=.4, ax=ax[1])
ax[1].set_title('Salary histogram by tenure: long-tenured people', fontsize='14');


The plots above show that long-tenured employees were not disproportionately comprised of higher-paid employees.

In [ ]:
# Create scatterplot of `average_monthly_hours` versus `last_evaluation`
plt.figure(figsize=(16, 9))
sns.scatterplot(data=df, x='average_monthly_hours', y='last_evaluation', hue='left', alpha=0.4)
plt.axvline(x=166.67, color='#ff6361', label='166.67 hrs./mo.', ls='--')
plt.legend(labels=['166.67 hrs./mo.', 'left', 'stayed'])
plt.title('Monthly hours by last evaluation score', fontsize='14');

From the scatterplot above, we can deduce that two categories of employees have resigned. The first group comprises overworked employees who have performed exceptionally well. In contrast, the second group consists of employees who have worked slightly below the nominal monthly average of 166.67 hours and have lower evaluation scores. There is a correlation between the number of hours worked and the evaluation score. The plot has few employees in the upper left quadrant, indicating that working long hours does not always guarantee a good evaluation score. Furthermore, most of the employees in this company work well over 167 hours per month.

In [ ]:
# Create plot to examine relationship between `average_monthly_hours` and `promotion_last_5years`
plt.figure(figsize=(16, 3))
sns.scatterplot(data=df, x='average_monthly_hours', y='promotion_last_5years', hue='left', alpha=0.4)
plt.axvline(x=166.67, color='#ff6361', ls='--')
plt.legend(labels=['166.67 hrs./mo.', 'left', 'stayed'])
plt.title('Monthly hours by promotion last 5 years', fontsize='14');

The plot above shows the following:

* very few employees who were promoted in the last five years left
* very few employees who worked the most hours were promoted
* all of the employees who left were working the longest hours

In [ ]:
# Create stacked histogram to compare department distribution of employees who left to that of employees who didn't
plt.figure(figsize=(11,8))
sns.histplot(data=df, x='department', hue='left', discrete=1, 
             hue_order=[0, 1], multiple='dodge', shrink=.5)
plt.xticks(rotation=45)
plt.title('Counts of stayed/left by department', fontsize=14);

There doesn't seem to be any department that differs significantly in its proportion of employees who left to those who stayed.

In [ ]:
# Keep only numeric columns 
numeric_cols = df_raw.select_dtypes(include=[np.number]).columns
df_num = df_raw[numeric_cols]

# Plot a correlation heatmap
plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(df_num.corr(), vmin=-1, vmax=1, annot=True, cmap=sns.color_palette("vlag", as_cmap=True))
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12);

The correlation heatmap shows a positive relationship between the number of projects, monthly hours, and evaluation scores. Additionally, an employee's satisfaction level negatively correlates with whether they decide to leave the company.

### Insights

Based on the observations, some workers quit their jobs due to unsatisfactory management. This decision is influenced by extended working hours, numerous projects, and decreased job contentment. Being overworked without receiving recognition or positive performance reviews can be disheartening. Additionally, a significant number of employees may be experiencing burnout. Interestingly, those who have worked at the company for over six years tend to stay.

## Model Building

- Determine which models are most appropriate
- Construct the model
- Confirm model assumptions
- Evaluate model results to determine how well your model fits the data


To figure out if an employee will leave the company, we need to predict a categorical outcome variable. This is known as a classification task, and in this case, it's binary classification. That means the outcome variable can be either 1 (indicating the employee left) or 0 (indicating the employee didn't leave). For this type of task, the most suitable models would be a Logistic Regression or a Tree-based Machine Learning mode


### Logistic Regression

**Logistic Regression model assumptions**
- Outcome variable is categorical
- Observations are independent of each other
- No severe multicollinearity among X variables
- No extreme outliers
- Linear relationship between each X variable and the logit of the outcome variable
- Sufficiently large sample size





In [ ]:
# Copy dataframe
df_enc = df.copy()

# Encode `salary` column as ordinal numeric category
salary_cat = ['low', 'medium', 'high']
df_enc['salary'] = (df_enc['salary'].astype('category').cat.set_categories(salary_cat).cat.codes)

# Dummy encode the `department` column
df_enc = pd.get_dummies(df_enc, drop_first=True)

# Display the new dataframe
df_enc.head()

Heatmap to visualize how correlated variables are.

In [ ]:
sub = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'tenure']
fig = plt.subplots(figsize=(8, 4))
sns.heatmap(df_enc[sub].corr(), annot=True, cmap="crest")
plt.title('Heatmap of the dataset')
plt.show()

**Outliers:**

Since logistic regression is quite sensitive to outliers, it would be a good idea at this stage to remove the outliers in the tenure column that were identified earlier.

In [ ]:
# Compute 25th percentile
percent_25 = df['tenure'].quantile(0.25)

# Compute 75th percentile
percent_75 = df['tenure'].quantile(0.75)

# Compute interquantile range
iqr = percent_75 - percent_25

# Define upper and lower limits for non-oulier values in column
upper_limit = percent_75 + 1.5 * iqr
lower_limit = percent_25 - 1.5 * iqr

In [ ]:
# Select rows without outliers in `tenure` and save resulting dataframe in a new variable
df_logreg = df_enc[(df_enc['tenure'] >= lower_limit) & (df_enc['tenure'] <= upper_limit)]

# Display first few rows of new dataframe
df_logreg.head()

### Data Splits

In [ ]:
# Separate the outcome variable from the features
y = df_logreg['left']
X = df_logreg.drop('left', axis=1)

# Slipt the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)


**Logistic Regression Model**

In [ ]:
# Construct and fit a logistic model
log_clf = LogisticRegression(random_state=42, max_iter=500).fit(X_train, y_train)

# Make predictions using the model
y_pred = log_clf.predict(X_test)

**Confusion Matrix**

In [ ]:
# Confusion Matrix
log_cm = confusion_matrix(y_test, y_pred, labels=log_clf.classes_)

# Display for the confusion matrix
log_disp = ConfusionMatrixDisplay(confusion_matrix=log_cm, display_labels=log_clf.classes_)

# Plot confusion matrix
log_disp.plot(values_format='')

# Display plot
plt.show()

The upper-left quadrant displays the number of true negatives. The upper-right quadrant displays the number of false positives. The bottom-left quadrant displays the number of false negatives. The bottom-right quadrant displays the number of true positives.

True negatives: The number of people who did not leave that the model accurately predicted did not leave.

False positives: The number of people who did not leave the model inaccurately predicted as leaving.

False negatives: The number of people who left that the model inaccurately predicted did not leave

True positives: The number of people who left the model accurately predicted as leaving

A perfect model would yield all true negatives and true positives, and no false negatives or false positives.

**Imbalance Check**

Checking the value counts in the `left` column. Since this is a binary classification task, the class balance informs the way you interpret accuracy metrics.

In [ ]:
df_logreg['left'].value_counts(normalize=True)

The split between the two classes is 83% to 17%, indicating that the data is not perfectly balanced but not too imbalanced. If the data was more severely imbalanced, resample it to achieve a more balanced distribution. However, in this instance, you can use the data without altering the class balance and proceed with model evaluation.

**Metrics Report**

In [ ]:
# Create classification report for logistic regression model
target_names = ['Predicted would not leave', 'Predicted would leave']
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)

# Convert to dataframe 
report_df = pd.DataFrame(report).transpose()

# Display results
report_df

We are going to tailor the dataframe so as to be the base for the metrics that will be used in the other models

In [ ]:
## Extracting relevant data from the report_df

# Extracting the last row
results = report_df.tail(1).reset_index(drop=True)

# Rename 'f1-score' column
results = results.rename(columns={'f1-score' : 'F1'})

# Adding a new column "model" at the beginning with sample values
results.insert(0, 'model', ['logistic regression'])

# Dropping the last column ("support") from the DataFrame
results.drop(columns=['support'], inplace=True)

# Adding 'accuracy' and 'auc' columns to the DataFrame
results['accuracy'] = report_df.loc['accuracy'][0]
results['AUC'] = roc_auc_score(y_test, y_pred)

# Display the `result` dataframe
results

The table above shows that the logistic regression model achieved a precision of 79%, recall of 82%, f1-score of 80%, and accuracy of 82% with and area under the curve (AUC) of 60%.

## Tree-based Model

This approach covers implementation of Decision Tree and Random Forest.

### Decision tree

In [ ]:
# Instantiate a decision tree model
tree = DecisionTreeClassifier(random_state=42)

# Dictionnary of hyperparameters
cv_params = {'max_depth' : [4, 6, 8, None],
             'min_samples_leaf' : [1, 2, 5],
             'min_samples_split' : [2, 4, 6]}

# Scoring
scoring = {'f1' : 'f1', 
           'accuracy' : 'accuracy', 
           'precision' : 'precision', 
           'recall' : 'recall', 
           'roc_auc' : 'roc_auc'}

# Instantiate GridSearch
tree_cv1 = GridSearchCV(tree, cv_params, scoring=scoring, cv=5, refit='roc_auc', verbose=True, n_jobs=-1) 


Fit the decision tree model to the training data.

In [ ]:
%%time
tree_cv1.fit(X_train, y_train)

In [ ]:
# Best parameters
tree_cv1.best_params_

In [ ]:
# Best score
tree_cv1.best_score_

This is a strong AUC score, which shows that this model can predict employees who will leave very well.

Next, you can write a function that will help you extract all the scores from the grid search.


In [ ]:
def make_results(model_name:str, model_object, metric:str):
    '''
    Arguments:
        model_name (string): what you want the model to be called in the output table
        model_object: a fit GridSearchCV object
        metric (string): precision, recall, f1, accuracy, or auc
  
    Returns a pandas df with the F1, recall, precision, accuracy, and auc scores
    for the model with the best mean 'metric' score across all validation folds.  
    '''

    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {'auc': 'mean_test_roc_auc',
                   'precision': 'mean_test_precision',
                   'recall': 'mean_test_recall',
                   'f1': 'mean_test_f1',
                   'accuracy': 'mean_test_accuracy'
                  }

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]

    # Extract Accuracy, precision, recall, and f1 score from that row
    auc = best_estimator_results.mean_test_roc_auc
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
  
    # Create table of results
    table = pd.DataFrame()
    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision],
                          'recall': [recall],
                          'F1': [f1],
                          'accuracy': [accuracy],
                          'AUC': [auc]
                        })
  
    return table

Use the function just defined to get all the scores from grid search.

In [ ]:
# Get all CV scores
tree1_cv_results = make_results('decision tree cv', tree_cv1, 'auc')

# Update `results` DataFrame
results = pd.concat([results, tree1_cv_results], axis=0)

# Sort 'results' by F1-score
results = results.sort_values(by='F1', ascending=False).reset_index(drop=True)

# Display `results` Dataframe
results

All of these scores from the decision tree model are strong indicators of good model performance.

Recall that decision trees can be vulnerable to overfitting, and random forests avoid overfitting by incorporating multiple trees to make predictions.

Define a function that gets all the scores from a model's predictions.

In [ ]:
def get_scores(model_name:str, model, X_test_data, y_test_data):
    '''
    Generate a table of test scores.

    In: 
        model_name (string):  How you want your model to be named in the output table
        model:                A fit GridSearchCV object 
        X_test_data:          numpy array of X_test data
        y_test_data:          numpy array of y_test data

    Out: pandas df of precision, recall, f1, accuracy, and AUC scores for your model
    '''

    preds = model.best_estimator_.predict(X_test_data)

    auc = roc_auc_score(y_test_data, preds)
    accuracy = accuracy_score(y_test_data, preds)
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)

    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision], 
                          'recall': [recall],
                          'F1': [f1],
                          'accuracy': [accuracy],
                          'AUC': [auc]
                         })
  
    return table

In [ ]:
# Get predictions from test data
tree_test_scores = get_scores('decision tree test', tree_cv1, X_test, y_test)

# Update `results` DataFrame
results = pd.concat([results, tree_test_scores], axis=0)

# Sort 'results' by F1-score
results = results.sort_values(by='F1', ascending=False).reset_index(drop=True)

# Display `results` Dataframe
results

**Tree Visualisation**

In [ ]:
# Extract column names
feature_names = list(X.columns)

# Class names
class_names = ['stayed', 'left']

# Plot the tree
plt.figure(figsize=(85,20))
plot_tree(tree_cv1.best_estimator_, 
          max_depth=6, fontsize=14, 
          feature_names=feature_names, 
          class_names=class_names, 
          filled=True);
plt.show()

**Feature Importance**

In [ ]:
tree_importances = pd.DataFrame(tree_cv1.best_estimator_.feature_importances_, 
                                 columns=['gini_importance'], 
                                 index=X.columns
                                )
tree_importances = tree_importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
tree_importances = tree_importances[tree_importances['gini_importance'] != 0]
tree_importances

In [ ]:
sns.barplot(data=tree_importances, x="gini_importance", y=tree_importances.index, orient='h')
plt.title("Decision Tree: Feature Importances for Employee Leaving", fontsize=12)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()

### Random forest

Construct a random forest model and set up cross-validated grid-search to exhuastively search for the best model parameters.

In [ ]:
# Instantiate model
rf = RandomForestClassifier(random_state=42)

# Hyperparameters
cv_params = {'max_depth' : [1, 3, 5, 10, None],
             'max_features' : [0.5, 1.0],
             'max_samples' : [0.5, 0.7, 1.0],
             'min_samples_leaf' : [1, 2, 5],
             'min_samples_split' : [2, 3, 5],
             'n_estimators' : [300, 500]}

# Scoring
scoring = {'accuracy' : 'accuracy',
           'precision' : 'precision',
           'f1' : 'f1',
           'recall' : 'recall',
           'roc_auc' : 'roc_auc'}

# Instantiate GridSearch
rf_cv1 = GridSearchCV(rf, cv_params, scoring=scoring, cv=5, refit='roc_auc', n_jobs=-1)

In [ ]:
%%time

# Fit the model on test data
rf_cv1.fit(X_train, y_train)

In [ ]:
import os

# Define a path to the folder where you want to save the model
path = os.path.join(os.getcwd(), 'model')

# Create the 'model/' directory if it doesn't exist
if not os.path.exists(path):
    os.makedirs(path)

Define functions to pickle the model and read in the model.

In [ ]:
def write_pickle(path, model_object, save_as:str):
    '''
    In: 
        path:         path of folder where you want to save the pickle
        model_object: a model you want to pickle
        save_as:      filename for how you want to save the model

    Out: A call to pickle the model in the folder indicated
    '''    

    # Create the full path for saving the model
    full_path = os.path.join(path, save_as + '.pickle')

    with open(full_path, 'wb') as to_write:
        pickle.dump(model_object, to_write)

In [ ]:
def read_pickle(path, saved_model_name:str):
    '''
    In: 
        path:             path to folder where you want to read from
        saved_model_name: filename of pickled model you want to read in

    Out: 
        model: the pickled model 
    '''
    
    # Create the full path for saving the model
    full_path = os.path.join(path, saved_model_name + '.pickle')

    with open(full_path, 'rb') as to_read:
        model = pickle.load(to_read)

    return model

In [ ]:
# Write pickle
write_pickle(path, rf_cv1, 'hr_rf1')

In [ ]:
# Read pickle
rf1 = read_pickle(path, 'hr_rf1')

Identify the best AUC score achieved by the random forest model on the training set.

In [ ]:
# Check best AUC score on CV
rf1.best_score_

Identify the optimal values for the parameters of the random forest model.

In [ ]:
# Check best params
rf1.best_params_

In [ ]:
# Get all CV scores
rf1_cv_results = make_results('random forest cv', rf1, 'auc')

# Update `results` DataFrame
results = pd.concat([results, rf1_cv_results], axis=0)

# Sort 'results' by F1-score
results = results.sort_values(by='F1', ascending=False).reset_index(drop=True)

# Display `results` Dataframe
results

In [ ]:
# Get predictions on test data
rf1_test_scores = get_scores('random forest test', rf1, X_test, y_test)

# Update `results` DataFrame
results = pd.concat([results, rf1_test_scores], axis=0)

# Sort 'results' by F1-score
results = results.sort_values(by='F1', ascending=False).reset_index(drop=True)

# Display `results` Dataframe
results

In [ ]:
# Pivot the `results` table
results_long = pd.melt(results, 'model')
results_long = results_long.rename(columns={'variable' : 'metric'})
results_long

**Comparing Models' results**

In [ ]:
# Initialize the figure
fig, ax = plt.subplots(2, 1, figsize=(12, 12))

# Barplot with Metrics on x-axis and new color scheme ('husl')
sns.barplot(x='metric', y='value', hue='model', data=results_long, ax=ax[0])

# Find the highest value for each metric to highlight the corresponding bar
for metric in results_long['metric'].unique():
    highest_value = results_long[results_long['metric'] == metric]['value'].max()
    # Highlight the bar with highest value for each metric
    for p in ax[0].patches:
        if p.get_height() == highest_value and p.get_x() >= list(results_long['metric'].unique()).index(metric) - 0.5:
            p.set_edgecolor('black')
            p.set_linewidth(1)

ax[0].set_title('Bar Chart of Model Evaluation Metrics (Metrics on x-axis)')
ax[0].set_ylabel('Value')
ax[0].set_xlabel('Metrics')

# Move legend to the bottom for the barplot
ax[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=5)

# Line Graph with Metrics on x-axis and new color scheme ('dark')
sns.lineplot(x='metric', y='value', hue='model', data=results_long, ax=ax[1], marker='o')
ax[1].set_title('Line Graph of Model Evaluation Metrics (Metrics on x-axis)')
ax[1].set_ylabel('Value')
ax[1].set_xlabel('Metrics')

# Remove legend from the line graph
ax[1].legend_.remove()

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**Model Choices Based on Metrics:**

`Random Forest Test`:Excels in Precision, F1 Score, and Accuracy. This model is less likely to falsely identify loyal employees as risks and offers an excellent general-purpose solution.

`Decision Tree Test`: Best in Recall. This model would be suitable if the company is more focused on capturing as many flight risks as possible, even at the risk of some false positives.

`Random Forest CV`:Best in AUC. This suggests that the model is excellent in differentiating between the classes. 


**Recommendation:**

Given the high costs associated with false positives (misidentifying loyal employees) and false negatives (failing to identify employees who will leave), a balanced approach might be best.
`Random Forest Test` is the most balanced model, excelling in precision, F1 score, and accuracy. It's less likely to misclassify employees, enabling the HR department to take targeted actions.

In [ ]:
# Generate array of values for confusion matrix
preds = rf1.best_estimator_.predict(X_test)
cm = confusion_matrix(y_test, preds, labels=rf1.classes_)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=rf1.classes_)
disp.plot(values_format='');

**interpretation:**

* High True Negative (TN): The model correctly identified a large number of employees who are not at risk of quitting or getting fired (TN = 2316). This is a strong indicator that the model is effective at ruling out employees who are not at risk.
* High True Positive (TP): The model also correctly identified a substantial number of employees who are at risk (TP = 434). This shows that the model is also effective at flagging employees who are actually at risk.
* Low False Positive (FP): The model incorrectly flagged only a small number of employees as being at risk when they are not (FP = 37). This could lead to unnecessary interventions but is relatively low.
* Low False Negative (FN): The model missed a very low number of employees who are at risk but were classified as not being at risk (FN = 5). This is also a good sign, as missing at-risk employees could have been detrimental.

Overall, this is a robust model for identifying employees who are and are not at risk of quitting or getting fired, with a low rate of false positives and false negatives.

**Feature Importance**

In [ ]:
# Get feature importances
feat_impt = rf1.best_estimator_.feature_importances_

# Get indices of top 10 features
ind = np.argpartition(rf1.best_estimator_.feature_importances_, -10)[-10:]

# Get column labels of top 10 features 
feat = X.columns[ind]

# Filter `feat_impt` to consist of top 10 feature importances
feat_impt = feat_impt[ind]

y_df = pd.DataFrame({"Feature":feat,"Importance":feat_impt})
y_sort_df = y_df.sort_values("Importance")
fig = plt.figure()
ax1 = fig.add_subplot(111)

y_sort_df.plot(kind='barh',ax=ax1,x="Feature",y="Importance")

ax1.set_title("Random Forest: Feature Importances for Employee Leaving", fontsize=12)
ax1.set_ylabel("Feature")
ax1.set_xlabel("Importance")

plt.show()

## Results and Evaluation

### Evaluation metrics

* **AUC** is the area under the ROC curve; it's also considered the probability that the model ranks a random positive example more highly than a random negative example.
* **Precision** measures the proportion of data points predicted as True that are actually True, in other words, the proportion of positive predictions that are true positives.
* **Recall** measures the proportion of data points that are predicted as True, out of all the data points that are actually True. In other words, it measures the proportion of positives that are correctly classified.
* **Accuracy** measures the proportion of data points that are correctly classified.
* **F1-score** is an aggregation of precision and recall.


## Model results

**1. Logistic Regression**

On the test set, the logistic regression model achieved the following:
- precision : 80%, 
- recall : 83%, 
- f1-score : 80% (all weighted averages), 
- accuracy : 83%, 
- AUC (area under the curve) : 60%

**2. Tree-based Machine Learning**

For both models, the test results were higher than the cross-validation's.

**Decision Tree**

On the test set, the decision tree model achieved the following:
- precision : 96%, 
- recall : 92%, 
- f1-score : 94% (all weighted averages), 
- accuracy : 98%, 
- AUC (area under the curve) : 96%

**Random Forest**

On the test set, the random forest model achieved the following:
- precision : 98%, 
- recall : 92%, 
- f1-score : 95% (all weighted averages), 
- accuracy : 98%, 
- AUC (area under the curve) : 95%

While the decision tree model has a higher AUC compared to the random forest, it can still be concluded that the random forest model is superior overall.


## Recommendations

To ensure that employees at the company are well-rested and remain satisfied, stakeholders should consider implementing the following recommendations.
- Limit the number of projects that employees can work on. 
- Promote employees who have been with the company for at least four years or investigate why these tenured employees are dissatisfied. 
- Stakeholders could either reward employees for working longer hours or limit their hours. 
- Inform employees about the company's overtime pay policies and clarify expectations around workload and time off. Company-wide and within-team discussions can also be held to address work culture. 
- Evaluation scores should be reserved for employees who work less than 200+ hours per month, and a proportionate scale should be used to reward employees who contribute more effort.